In [ ]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq

In [ ]:
client, llm = init_groq()

In [ ]:
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"

In [12]:
my_tools = []

credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    
    GoogleDocWriteTool(credentials_path)
)

my_tools.append(
    
    GoogleDriveUploadTool(credentials_path)
)

my_tools.append(
    
    GoogleSheetsUpdateTool(credentials_path)
)

my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

my_tools.append(
    ImprovedSearchTool(credentials_path)
)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65376%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=BJhgn9raiFDkbA45LgAZl3TjapRL4q&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65384%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=VvzEAMslNWvx90wa4rxQsmyX8zh5IO&access_type=offline
Authentication successful.


In [13]:
from langchain.agents import load_tools
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]
my_tools.append(humantool)

In [14]:
# Chat history tracking
chat_history = []

In [15]:
chat_history

[]

In [16]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.

RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

You need credentials of the user. Here they are:
{name}
{email}
{recipient_email}
{phone}
-------
HERE IS PATH to documents:
{thepath}
-------
Here are the tools you can use. DO NOT TRY TO USE ANYTHING OTHER THAN THESE:
{tools}
-------
HERE IS GUIDANCE ON EXACTLY WHAT TO PASS IN TO THE TOOLS:
GoogleDocWriteTool: INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! write--> a boolean true or false --> append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: GENERATE THE WRITABLE CONTENT YOURSELF!!!! | APPEND: decide whether or not you want to append to document (must be in lowercase)
MoveFileTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the response and then pass it into tool: {input} ; NEVER PASS IN 'root' UNLESS USER INDICATES to move file into 'google drive' or 'my drive'; for rest of the parameters, there will be instructions given over time for you.
GoogleSheetsUpdateTool: Input ONLY ONLY these into the tool PROPERLY! --> {name}, {phone}
CreateFolderTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in response --> {input}; pass in 'root' as the value for parent_folder_id parameter if user indicates 'google drive' or 'my drive' ||| PLEASE OUTPUT THE EXACT NAME OF THE FOLDER THAT WAS CREATED
FolderMovementTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the response and then pass it into tool: {input} ; NEW_PARENT_FOLDER_NAME also deduce from response and PASS IT IN {input}. SPECIAL CASE - PASS IN 'root' if user says move into google drive or my drive: {input}. The IDs you will find out later. for now they are optional.
FileOrganizerTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> PARENT_FOLDER_NAME please deduce from the response and then pass it into tool - PASS IN 'root' if user says move into google drive or my drive: {input}
GoogleDriveUploadTool: INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {email}
GmailSendPdfTool: IF THERE IS A FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}
GmailSendPdfTool: IF THERE IS NO FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing
ImprovedSearchTool: DEDUCE THE FILE NAME OR FOLDER NAME from response --> {input}, and pass it in to this tool; AND if you know the ID you can pass it in as well.

IMPORTANT: 1. THIS IS FOR ANYTHING INVOLVING MOVING FILES OR FOLDERS:
    - MAKE SURE THAT ID'S ARE 100% CORRECT! IF ID'S ARE WRONG, LIFE WILL END AS WE KNOW IT!
   - First, use the ImprovedSearchTool to find the file ID for the file name provided.
   - Then, use the ImprovedSearchTool to find the folder ID for the folder name provided.
   - Once you have both IDs, use the MoveFileTool to move the file to the specified folder.

IMPORTANT: THIS IS FOR ORGANIZING FILES IN "MY DRIVE" (ROOT) OR ANY GIVEN FOLDER 
    - NEVVVVVEEERRRR USE CREATE FOLDER TOOL HERE!!!!!
    - YOU MUST USE IMPROVED SEARCH TOOL TO SEARCH FOR FOLDERS OTHER THAN [ROOT/MY DRIVE/GOOGLE DRIVE]
    - ALWAYS USE THE FILE ORGANIZER TOOL FOR THIS TASK.

IMPORTANT: THIS IS FOR GOOGLE DOC STUFF:
    - IF USER TELLS YOU TO ONLY CREATE GOOGLE DOC (NOTHING ELSE AFTER IT), pass in empty string into 'input_text' PARAMETER 
    - OTHERWISE YOU MUST WRITE TO DOC - 'write' PARAMETER AS 'True'

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.


---------
Chat History: {chat_history}
User Input: {input}
"""



# User credentials
credentials = {
    "name": "Sriram Nallani",
    "email": "sriramnallani35@gmail.com",
    "recemail": "thegreatchrisjohn@gmail.com",
    "phone": "4973993399"
}

# Google TTS client initialization
def synthesize_speech(text, output_file):
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')

async def stream_agent_output(agent_executor:AgentExecutor):
    async for event in agent_executor.astream_events({"input": "Your query here"}):
        yield f"data: {json.dumps(event)}\n\n"

agent = None
# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    input_text += "YOU MUST STIKE UP A CONVERSATION WITH HUMAN. DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT. DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.DO NOT CREATE FOLDERS WHEN ORGANIZING FILES. UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER! ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL\n" + template + "\n"
    thepath = "C:\\DEV\\Webdevfolder\\realestateai\\documents\\renovationcontract.pdf"
    input_text += ("\n\n Here is extra info you will need: \nCredentials:\n" + str(credentials) + "\n"
                   + "Path to file on local system:\n" + thepath)

    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    from langchain.agents.structured_chat.prompt import FORMAT_INSTRUCTIONS
    prompt_template = PromptTemplate.from_template(
        template="If you say or do anything other than what is in here, life will end as we know it. \n\n" + FORMAT_INSTRUCTIONS + "\n\n" + template
    )

    tools_str = '\n'.join([f"{tool.name}: {tool.description}" for tool in my_tools])
    prompt_template.format(input = input_text,
        chat_history = chat_history,
        tools = tools_str,
        tool_names = tools_str,
        name = credentials['name'],
        email = credentials['email'],
        recipient_email = credentials['recemail'],
        phone = credentials['phone'],
        thepath = "C:\\Users\\pc-user1\\Downloads\\Okati okati okati.mp3"
    )
    
    #print('PROMPT TEMP\n',prompt_template)

    agent = initialize_agent(
        tools=my_tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        prompt_template=prompt_template,
        verbose=True,
        max_iterations=1000
    )
    
    response = agent.invoke({
        "input": input_text,
        "chat_history": chat_history,
        "tools": tools_str,
        "name": credentials['name'],
        "email": credentials['email'],
        "recipient_email": credentials['recemail'],
        "phone":credentials['phone'],
        "thepath": thepath
    })
    chat_history.append({"input": input_text, "response": response})
    
    # Use Google TTS to convert the response to speech
    #tts_output_file = "output.mp3"
    #synthesize_speech(response, tts_output_file)
    #print(f'Response audio saved to {tts_output_file}')
    
    return response

# Can you please write a 5 paragraph essay about smartphones and their relation to artificial intelligence, 
# and then move the document to a folder related to 'FCPS'. 
# Then create a folder named 'dummy folder 2' and then move that to folder called 'LM'.\
# Please upload the file to google drive. move it into Audio_MyDrive folder.
#Locate the folder called JUST THE LOWERCASE LETTER 'a' ! PLEASE ANALYZE IMPROVED SEARCH TOOL RESULTS AND GIVE ME INFO.
# thepath= "C:\\Users\\pc-user1\\Downloads\\Okati okati okati.mp3"
#Add the following to the document called 'Project Plan': 'This is the project plan for Q3. Key milestones include development, testing, and deployment.'

cc_out = """
**RENOVATION QUOTE DOCUMENT**

**Project Address:** 13599 Cobra Dr, Herndon, VA 20171

**Project Overview:**
Project Description: Basement renovation project with modern design and premium finishes.
Length of Time for Project: Completion within 2 months.

**Cost Breakdown:**

1. **Labor Costs:**
    - Total Labor Cost: $11,250

2. **Material Costs:**
    - Total Material Cost: $68,500
    - Itemized Costs:
        + Flooring: $4,500
        + Lighting: $3,000
        + Shelving: $6,500
        + Electrical and HVAC: $5,000
        + Walls and Ceiling: $3,500
        + Games Area: $7,000
        + Tinted Glass Wall: $10,000
        + Hot Tub: $15,000
        + Miscellaneous: $14,000

**Total Estimated Cost:**
    - Total Cost of Labor: $11,250
    - Total Cost of Materials: $68,500
    - **Grand Total: $79,750**

**Payment Terms:**
    - Deposit Required: 20% ($15,950.00)
    - Payment Schedule: Monthly payments of $15,950 for 4 months
    - Final Payment Due: Upon completion of the project
"""
#

user_input = f""" 
hello! how are you doing. Please take this input and WRITE IT EXACTLY into a google doc. ELIMINATE ALL THE WEIRD CHARACTERS like slash n and stuff. Name the doc appropriately. 
    
""" +cc_out 

# + """
#    \n THEN, please upload the renovationcontract.pdf document to google drive and update google sheet with the name "Gautham", phone "5715996302" and LINKS IN THIS EXACT ORDER --> link to the document of the renovation quote you just created and the link to uploaded pdf.
# """
result = run_agent(user_input)
print(result['output'])




> Entering new AgentExecutor chain...
Thought: I understand the user's request to write the input exactly into a Google Doc and eliminate any weird characters.

Action:
```
{
  "action": "GoogleDocWriteTool",
  "action_input": {
    "input_text": "**RENOVATION QUOTE DOCUMENT**\n\n**Project Address:** 13599 Cobra Dr, Herndon, VA 20171\n\n**Project Overview:**\nProject Description: Basement renovation project with modern design and premium finishes.\nLength of Time for Project: Completion within 2 months.\n\n**Cost Breakdown:**\n\n1. **Labor Costs:**\n    - Total Labor Cost: $11,250\n\n2. **Material Costs:**\n    - Total Material Cost: $68,500\n    - Itemized Costs:\n        + Flooring: $4,500\n        + Lighting: $3,000\n        + Shelving: $6,500\n        + Electrical and HVAC: $5,000\n        + Walls and Ceiling: $3,500\n        + Games Area: $7,000\n        + Tinted Glass Wall: $10,000\n        + Hot Tub: $15,000\n        + Miscellaneous: $14,000\n\n**Total Estimated Cost:**\n    -

KeyboardInterrupt: 